2. Event Study Metrics Calculations

This notebook computes abnormal returns (AR), cumulative abnormal returns (CAR), and other event-study metrics for each IPO in the sample.

In [4]:

import pandas as pd
import os
sample1 = pd.read_csv("clean_sample1.csv")
sample2 = pd.read_csv("clean_sample2.csv")
nifty = pd.read_csv("clean_nifty_returns.csv")

sample1.head()
sample2.head()
nifty.head()

,date,close,nifty_return
0,2023-12-20,21475,NaN
1,2023-12-21,21502,0.001257
2,2023-12-22,21488,-0.000651
3,2023-12-23,21515,0.001257
4,2023-12-24,21542,0.001255


In [43]:
import pandas as pd
import os
sample1 = pd.read_csv("clean_sample1.csv")
sample2 = pd.read_csv("clean_sample2.csv")
nifty = pd.read_csv("clean_nifty_returns.csv")

#Removing the first NaN row because pct_change will naturally create one NaN
sample1 = sample1.dropna()
sample2 = sample2.dropna()
 

2.1 Calculation of Abnormal Return and Cumulative Abnormal Return

(a) Abnormal Return :  ARt​= RtIPO​−RtNIFTY​

(b) Cumulative Abnormal Return: CARt1​,t2​​=t=t1​∑t2​​ARt​

(c) Compute AR per IPO

(d) Compute CAR for the event window

(e) Validate results with summary checks

In [54]:
#Abnormal Return Calculation
sample1["abnormal_return"] = sample1["ipo_return"] - sample1["nifty_return"]
sample2["abnormal_return"] = sample2["ipo_return"] - sample2["nifty_return"]

#CAR Calculation (Cumulative Abnormal Return)
sample1["car"] = sample1['abnormal_return'].cumsum()
sample2["car"] = sample2['abnormal_return'].cumsum()
 


,date,close,ipo_return,nifty_return,abnormal_return,car
1,2024-01-02,104,0.019608,0.000691,0.018917,0.018917
2,2024-01-03,103,-0.009615,-0.000460,-0.009155,0.009762
3,2024-01-04,105,0.019417,0.001520,0.017898,0.027660
4,2024-01-05,108,0.028571,0.001104,0.027468,0.055127
5,2024-01-08,110,0.018519,0.001696,0.016823,0.071950


2.2 Calculation of All Metrics

This section includes calculation of various relevant metrics computed via formulas implemented directly in Python

In [2]:
def compute_30_day_metrics(df):
    df_30 = df.head(30)             #First 30 days after listing

    mean_return = df_30['ipo_return'].mean()
    mean_abnormal_30 = df_30["abnormal_return"].mean()
    car_30 = df_30["car"].iloc[-1]
    volatility_return_30 = df_30['ipo_return'].std()
    volatility_abnormal = df_30["abnormal_return"].std()

    #beta calculation
    beta = df_30['ipo_return'].cov(df_30['nifty_return']) / df_30['nifty_return'].var()

    #Sharpe Like Metric but without Risk-free rate
    sharpe_like = mean_return/volatility_return_30 if volatility_return_30 != 0 else None

    #Max_Drawdown
    cum_curve = (1 + df_30['ipo_return']).cumprod()
    running_max = cum_curve.cummax()
    drawdown = (cum_curve - running_max) / running_max
    max_drawdown = drawdown.min()

    return {
    "mean_return": mean_return,
    "mean_abnormal_30": mean_abnormal_30,
    "car_30": car_30,
    "volatility_return_30": volatility_return_30,
    "volatility_abnormal": volatility_abnormal,
    "beta": beta,
    "sharpe_like": sharpe_like,
    "max_drawdown": max_drawdown
}


sample1_metrics = compute_30_day_metrics(sample1)
sample2_metrics = compute_30_day_metrics(sample2)
print (sample1_metrics)
print (sample2_metrics)

{'mean_return': np.float64(0.01646627623624749), 'mean_abnormal_30': np.float64(0.015630431752341144), 'car_30': np.float64(0.14067388577107032), 'volatility_return_30': 0.018059501648673135, 'volatility_abnormal': 0.017204581295557363, 'beta': np.float64(17.435858126734892), 'sharpe_like': np.float64(0.911779104239972), 'max_drawdown': -0.01769911504424772}
{'mean_return': np.float64(0.012090668953962522), 'mean_abnormal_30': np.float64(0.011254824470056178), 'car_30': np.float64(0.10129342023050561), 'volatility_return_30': 0.011643540248923749, 'volatility_abnormal': 0.011439452894603411, 'beta': np.float64(3.1464102771212685), 'sharpe_like': np.float64(1.0384014393800978), 'max_drawdown': -0.008888888888888839}


2.2 Comparision of Metrics

In [41]:
#COMPARISIONS
sample1_summary = {
    'mean_return': 0.01646627623624749,
    'mean_abnormal_30': 0.015630431752341144,
    'car_30': 0.14067388577107032,
    'volatility_return_30': 0.018059501648673135,
    'volatility_abnormal': 0.017204581295557363,
    'beta': 17.435858126734892,
    'sharpe_like': 0.911779104239972,
    'max_drawdown': -0.01769911504424772
}

sample2_summary = {
    'mean_return': 0.012090668953962522,
    'mean_abnormal_30': 0.011254824470056178,
    'car_30': 0.10129342023050561,
    'volatility_return_30': 0.011643540248923749,
    'volatility_abnormal': 0.011439452894603411,
    'beta': 3.1464102771212685,
    'sharpe_like': 1.0384014393800978,
    'max_drawdown': -0.008888888888888839
}


import pandas as pd
comparision = pd.DataFrame({ "Sample_1": sample1_summary, "Sample_2": sample2_summary})
comparision

#With a Difference column
comparision['Difference(S1 - S2)'] = comparision["Sample_1"] - comparision["Sample_2"]
comparision

 


,Sample_1,Sample_2,Difference(S1 - S2)
mean_return,0.016466,0.012091,0.004376
mean_abnormal_30,0.015630,0.011255,0.004376
car_30,0.140674,0.101293,0.039380
volatility_return_30,0.018060,0.011644,0.006416
volatility_abnormal,0.017205,0.011439,0.005765
beta,17.435858,3.146410,14.289448
sharpe_like,0.911779,1.038401,-0.126622
max_drawdown,-0.017699,-0.008889,-0.008810


2.3 Exporting Final Files

In [55]:
# Save final processed DataFrames to the existing Processed_Data folder
output_path_sample1 = "../Data/Processed_Data/final_sample1.csv"
output_path_sample2 = "../Data/Processed_Data/final_sample2.csv"

sample1.to_csv(output_path_sample1, index=False)
sample2.to_csv(output_path_sample2, index=False)
